In [15]:
%matplotlib inline
import pandas as pd
import collections
import itertools
import math
import random
import warnings
import time
warnings.filterwarnings('ignore')

In [17]:
data = [
    ["Real", "La Liga", "Madrid", "Espana"],
    ["Athletico", "La Liga", "Madrid", "Spain"],
    ["Barcleona F.C", "La Liga", "Barcelona", "Spain"],
    ["Athletico", "Spanish_League", "Madrid", "Spain"],
    ["Real", "La Liga", "Madrid", "Spain"]
]

df = pd.DataFrame(data, columns=["Team", "League", "City", "Country"])

In [18]:
# data = [
#     ["birmingham", "jefferson"],
#     ["sheffield", "jefferson"],
#     ["sheffield", "jefferson"],
#     ["birmixgham", "jefferson"],
#     ["birmingham", "jefferson"],
#     ["sheffield", "jeffxrson"],
#     ["sheffield", "jefferson"],
#     ["sheffield", "jefferson"],
#     ["birmingham", "jefferson"],
# ]

# df = pd.DataFrame(data, columns=["City", "County"])

In [19]:
df

,Team,League,City,Country,Year,Rank
0,Barcelona F.C.,La Liga,Barcelona,Spain,2019,1
1,Real,La Liga,Madrid,Espana,2019,3
2,Athletico,La Liga,Madrid,Spain,2019,2
3,Barcelona F.C.,Barcelona,Catalonia,La Liga,2018,1
4,Athletico,Spanish_League,Madrid,Spain,2018,2
5,Real,La Liga,Madrid,Spain,2018,3


In [20]:
constraints = [1, 2, 3]

In [21]:
def algo(df, constraints):
    df_c = df.copy()
    if 1 in constraints:
        for i, r in df_c.iterrows():
            if r.Team != "NULL" and r.City != "NULL":
                errors = df_c[(df_c.Team == r.Team) & (df_c.City != r.City) & (df_c.City != "NULL")]
                if errors.shape[0] > 0:
                    c = collections.Counter(df_c[df_c.Team == r.Team].City)
                    if "NULL" in c:
                        c.pop("NULL")
                    if len(c) > 0:
                        if c.most_common(1)[0][1] > 1:
                            df_c["City"][i] = c.most_common(1)[0][0] 

    if 2 in constraints:
        for i, r in df_c.iterrows():
            if r.League != "NULL" and r.Country != "NULL":
                errors = df_c[(df_c.League == r.League) & (df_c.Country != r.Country) & (df_c.Country != "NULL")]
                if errors.shape[0] > 0:
                    c = collections.Counter(df_c[df_c.League == r.League].Country)
                    if "NULL" in c:
                        c.pop("NULL")
                    if len(c) > 0:
                        if c.most_common(1)[0][1] > 1:
                            df_c["Country"][i] = c.most_common(1)[0][0]
                                   
                        
    if 3 in constraints:
        for i, r in df_c.iterrows():
            if r.City != "NULL" and r.Country != "NULL":
                errors = df_c[(df_c.City == r.City) & (df_c.Country != r.Country) & (df_c.Country != "NULL")]
                if errors.shape[0] > 0:
                    c = collections.Counter(df_c[df_c.City == r.City].Country)
                    if "NULL" in c:
                        c.pop("NULL")
                    if len(c) > 0:
                        if c.most_common(1)[0][1] > 1:
                            df_c["Country"][i] = c.most_common(1)[0][0]


    
    return df_c

In [9]:
algo(df, [1, 2, 3])

,Team,League,City,Country
0,Barcleona F.C,La Liga,Barcelona,Spain
1,Real,La Liga,Madrid,Spain
2,Athletico,La Liga,Madrid,Spain
3,Athletico,Spanish_League,Madrid,Spain
4,Real,La Liga,Madrid,Spain


In [10]:
df

,Team,League,City,Country
0,Barcleona F.C,La Liga,Barcelona,Spain
1,Real,La Liga,Madrid,Espana
2,Athletico,La Liga,Madrid,Spain
3,Athletico,Spanish_League,Madrid,Spain
4,Real,La Liga,Madrid,Spain


In [11]:
import time
df = df
constraints = [1, 2, 3]
df_copy = df.copy()
cells = list(itertools.product(df.index, list(df_copy.columns)))

start = time.time()

results_shapp = {}
for i in range(len(cells) + 1):
    for comb in itertools.combinations(cells, i):
        df_c2 = df_copy.copy()
        for cell in cells:
            if cell not in comb:
                df_c2[cell[1]][cell[0]] = "NULL"
        df_repair = algo(df_c2, constraints)
        results_shapp[tuple(sorted(comb))] = int(df_repair["Country"][1]=="Spain")
    print("Done with len " + str(i))
print(time.time() - start)

Done with len 0
Done with len 1
Done with len 2
Done with len 3
Done with len 4
Done with len 5
Done with len 6
Done with len 7
Done with len 8
Done with len 9
Done with len 10
Done with len 11
Done with len 12
Done with len 13
Done with len 14
Done with len 15
Done with len 16
Done with len 17
Done with len 18
Done with len 19
Done with len 20
9602.089573860168


In [ ]:
import time
# df = df
# constraints = [1, 2, 3]

df = pd.read_csv('./testdata/La_Liga.csv')
constraints_path = './testdata/La_liga_constraints.txt'


df_copy = df.copy()
cells = list(itertools.product(df.index, list(df_copy.columns)))

start = time.time()

results_shapp = {}
for i in range(len(cells) + 1):
    for comb in itertools.combinations(cells, i):
        df_c2 = df_copy.copy()
        for cell in cells:
            if cell not in comb:
                df_c2[cell[1]][cell[0]] = "NULL"
        df_repair = algo(df_c2, constraints)
        results_shapp[tuple(sorted(comb))] = int(df_repair["Country"][0]=="Spain")
    print("Done with len " + str(i))
print(time.time() - start)

In [12]:
cells = list(itertools.product(df.index, list(df.columns)))
# cells.remove((0, 'Country'))

for cell_i in cells:
    shapley_value = 0
    for comb in results_shapp:
        if cell_i not in comb:
            comb_i = tuple(sorted(tuple((comb + (cell_i,)))))
            factor = math.factorial(len(comb)) * math.factorial(len(cells) - len(comb) - 1)
            factor /= math.factorial(len(cells))
            if results_shapp[comb_i] and not results_shapp[comb]:
                shapley_value += factor*(results_shapp[comb_i] - results_shapp[comb])
    print(cell_i, shapley_value * 10000)    

(0, 'Team') 0
(0, 'League') 423.07692307693003
(0, 'City') 0
(0, 'Country') 423.07692307693003
(1, 'Team') 0
(1, 'League') 796.8142968141975
(1, 'City') 796.8142968141975
(1, 'Country') 3296.8142968143375
(2, 'Team') 0
(2, 'League') 372.5718725718877
(2, 'City') 372.5718725718877
(2, 'Country') 963.4809634808998
(3, 'Team') 0
(3, 'League') 0
(3, 'City') 423.07692307693003
(3, 'Country') 423.07692307693003
(4, 'Team') 0
(4, 'League') 372.5718725718877
(4, 'City') 372.5718725718877
(4, 'Country') 963.4809634808998


In [13]:
df

,Team,League,City,Country
0,Barcleona F.C,La Liga,Barcelona,Spain
1,Real,La Liga,Madrid,Espana
2,Athletico,La Liga,Madrid,Spain
3,Athletico,Spanish_League,Madrid,Spain
4,Real,La Liga,Madrid,Spain


In [41]:
cells

[(0, 'League'),
 (0, 'City'),
 (0, 'Country'),
 (1, 'League'),
 (1, 'City'),
 (1, 'Country'),
 (2, 'League'),
 (2, 'City'),
 (2, 'Country'),
 (3, 'League'),
 (3, 'City'),
 (3, 'Country'),
 (4, 'League'),
 (4, 'City'),
 (4, 'Country'),
 (5, 'League'),
 (5, 'City'),
 (5, 'Country')]